In [0]:
# Install the required module
%pip install flask

import mlflow
import mlflow.deployments
import pandas as pd
import mlflow.metrics.genai

# Set the model registry URI
mlflow.set_registry_uri("databricks-unity-catalog")

mlflow.deployments.set_deployments_target("databricks")

client = MlflowClient()
models = client.search_registered_models()
for model in models:
    print(model.name)

eval_data = pd.DataFrame(
    {
        "input": [
            "What is the capital of France?",
            "What is the capital of Germany?",
            "What is the capital of Spain?"],
        "ground_truth": ["Paris", "Berlin", "Madrid"]}
)

MODELS = ["bge-large-en"]

max_tokens = 100
temperature = 0.0

for model in MODELS:
    with mlflow.start_run() as run:
        mlflow.log_params({
            "model": model,
            "max_tokens": max_tokens,
            "temperature": temperature
        })

        results = mlflow.evaluate(
            model=f"models:/{model}/latest",
            data=eval_data,
            targets="ground_truth",
            extra_metrics=[
                mlflow.metrics.genai.answer_correctness(),
                mlflow.metrics.genai.answer_relevance(),
                mlflow.metrics.genai.answer_similarity()
            ],
            model_type="llm"
        )

        mlflow.log_metrics(results.metrics) 
        display(results.metrics)

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


/databricks/python/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.4/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-4966053403184466>, line 41
     34 with mlflow.start_run() as run:
     35     mlflow.log_params({
     36         "model": model,
     37         "max_tokens": max_tokens,
     38         "temperature": temperature
     39     })
---> 41     results = mlflow.evaluate(
     42         model=f"models:/{model}/latest",
     43         data=eval_data,
     44         targets="ground_truth",
     45         extra_metrics=[
     46             mlflow.metrics.genai.answer_correctness(),
     47             mlflow.metrics.genai.answer_relevance(),
     48             mlflow.metrics.genai.answer_similarity()
     49         ],
     50         model_type="llm"
     51     )
     53     mlflow.log_metrics(results.metrics) 
     54     display(results.metrics)

File /databricks/python/lib/python3.11/site-packages/mlflow/models/evalua

In [0]:
from mlflow import MlflowClient

client = MlflowClient()
models = client.search_registered_models()
for model in models:
    print(model.name)

### Need to specify the exact model name